In [1]:
def trace(func):
    
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    
    inner.__module__ = func.__module__
    inner.__name__ = func.__name__
    inner.__doc__ = func.__doc__
    return inner

In [2]:
@trace
def identity(x):
    "I do nothing useful."
    return x

In [3]:
identity(42)

identity (42,) {}


42

In [4]:
?identity

Signature: identity(*args, **kwargs)
Docstring: I do nothing useful.
File:      ~/ME/computer-science/CSC_Python/<ipython-input-1-f32747a5d797>
Type:      function


Module where the function was defined:

In [5]:
identity.__name__, identity.__doc__, identity.__module__

('identity', 'I do nothing useful.', '__main__')

The same decorator saving original attributes using function from `functools` module:

In [6]:
import functools


def trace(func):
    
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    
    functools.update_wrapper(inner, func)
    return inner


@trace
def identity(x):
    "I do nothing useful."
    return x

In [7]:
identity.__name__, identity.__doc__, identity.__module__

('identity', 'I do nothing useful.', '__main__')

In [8]:
def trace(func):
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    
    return inner


@trace
def identity(x):
    "I do nothing useful."
    return x

In [9]:
identity.__name__, identity.__doc__, identity.__module__

('identity', 'I do nothing useful.', '__main__')

Global variable `trace_enabled` to enable/disable `trace`:

In [10]:
trace_enabled = False


def trace(func):
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    
    return inner if trace_enabled else func

Decorator with argument:

In [11]:
def trace(handle):
    
    def decorator(func):
        
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs, file=handle)
            return func(*args, **kwargs)
        
        return inner
    
    return decorator

In [12]:
import sys


@trace(sys.stderr)
def identity(x):
    "I do nothing useful."
    return x


identity(42)

identity (42,) {}


42

Generalizing decorators:

In [13]:
def with_arguments(deco):
    
    @functools.wraps(deco)    
    def wrapper(*dargs, **dkwargs):
        
        def decorator(func):
            result = deco(func, *dargs, **dkwargs)
            functools.update_wrapper(result, func)
            return result
        
        return decorator
    
    return wrapper        

In [14]:
@with_arguments
def trace(func, handle):
    
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs, file=handle)
        return func(*args, **kwargs)
    
    return inner


@trace(sys.stderr)
def identity(x):
    return x

identity(42)

identity (42,) {}


42

Simpler:

In [15]:
def trace(func=None, *, handle=sys.stdout):
    
    if func is None:
        return lambda func: trace(func, handle=handle)
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs, file=handle)
        return func(*args, **kwargs)
    return inner


@trace(handle=sys.stderr)
def identity(x):
    return x

identity(42)

identity (42,) {}


42

### Decorator practice

Timer:

In [16]:
import time

In [17]:
def timethis(func=None, *, n_iter=100):

    if func is None:
        return lambda func: timethis(func, n_iter=n_iter)
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")
        acc = float("inf")
        for i in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)
        return result
    return inner

In [18]:
result = timethis(sum)(range(10 ** 7))

sum ... 0.09708845799997334


Profiling:

In [19]:
def profiled(func):
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    
    inner.ncalls = 0
    return inner

In [20]:
@profiled
def identity(x):
    return x

In [21]:
identity(42)

42

In [22]:
identity.ncalls

1

One-time usage:

In [23]:
def once(func):    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        if not inner.called:
            func(*args, **kwargs)
            inner.called = True
    inner.called = False
    return inner

@once
def initialize_settings():
    print('Settings initialized')

initialize_settings()
initialize_settings()

Settings initialized


Memoization:

In [24]:
def memoized(func):
    cache = {}
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items()))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return inner

Deprecated:

In [25]:
import warnings


def deprecated(func):
    code = func.__code__
    warnings.warn_explicit(
        func.__name__ + " is deprecated.",
        category=DeprecationWarning,
        filename=code.co_filename,
        lineno=code.co_firstlineno + 1
    )
    return func

In [26]:
@deprecated
def identity(x):
    return x

In [27]:
identity(42)

42

Contracts:

In [28]:
def pre(cond, message):
    
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            assert cond(*args, **kwargs), message
            return func(*args, **kwargs)
        return inner
    return wrapper

In [29]:
@pre(lambda x: x >= 0, "negative argument")
def checked_log(x):
    pass

In [30]:
checked_log(-42)

AssertionError: negative argument

In [31]:
import math

def post(cond, message):
    
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            result = func(*args, **kwargs)
            assert cond(result), message
            return result
        return inner
    return wrapper

@post(lambda x: not math.isnan(x), "not a number")
def something_useful():
    return float("nan")

In [32]:
something_useful()

AssertionError: not a number

Several decorators:

In [33]:
def square(func):
    return lambda x: func(x * x)

def addsome(func):
    return lambda x: func(x + 42)

In [34]:
@square
@addsome
def identity(x):
    return x

In [35]:
identity(2)

46

### Functools

Generalized functions (like `len`):

In [37]:
@functools.singledispatch
def pack(obj):
    type_name = type(obj).__name__
    assert False, "Unsupported type: " + type_name
    
@pack.register(int)
def _(obj):
    return b"I" + hex(obj).encode("ascii")

@pack.register(list)
def _(obj):
    return b"L" + b",".join(map(pack, obj))

In [38]:
pack([1, 2, 3])

b'LI0x1,I0x2,I0x3'

In [39]:
pack(42.)

AssertionError: Unsupported type: float